In [17]:
import numpy as np 
import pandas as pd 
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK

### Minimize the total cost of production over the 6 month time span where total cost of production is equal to:
### sum(Units Produced * Unit Production Cost) + sum(Holding Cost * Unit Production Cost * Average Inventory)

In [224]:
huge_number = float("inf")

months = 6
start_inventory = 2750
inventory_capacity = 6000
safety_stock = 1500
production_capacity_max = np.array([4000, 3500, 4000, 4500, 4000, 3500])
production_capacity_min = np.array([2000, 1750, 2000, 2250, 2000, 1750])

#setup_cost = 3.
unit_production_cost = np.array([240, 250, 265, 285, 280, 260])
holding_cost = 0.015

demand = np.array([1000, 4500, 6000, 5500, 3500, 4000])

In [225]:

start_values = []
final_production_cost = huge_number
for m in range(months): #Iterate over each month in the production plan
    for i in range(production_capacity_min[m], production_capacity_max[m]+1): 
        #iterate over possible production values between minimum production and max production
        units_produced = production_capacity_max.copy() #set the starting production values to the maximum values
        units_produced[m] = i #replace the production value with the value for the month being evaluated
        initial_inventory = [start_inventory]
        #calculate the final ending inventory with the given values
        final_ending_inventory = sum(units_produced) \
                                + start_inventory \
                                - sum(demand)

        #if the final ending inventory meets the minimum safety stock value then determine the total cost
        if final_ending_inventory >= safety_stock:
            ending_inventory = []
            average_inventory = []
            for k in range(stages):
                #determine the inventory values for each month
                ending_inventory.append(initial_inventory[k]+units_produced[k]-demand[k])
                average_inventory.append((initial_inventory[k]+ending_inventory[k])/2)
                if k< stages-1:
                    initial_inventory.append(ending_inventory[k])
            production_cost = units_produced * unit_production_cost + (holding_cost*unit_production_cost*average_inventory)
            #if the production cost is less than any previous production cost, then record the value as the lowest cost
            if sum(production_cost) < final_production_cost:
                final_production_cost = sum(production_cost)
                start_values = ([units_produced, final_production_cost])



In [ ]:
print(f'Initial Inventory Values: {start_values[0]}')
print(f'Total Cost: ${start_values[1]:,.0f}')


In [19]:
production_constraints = pd.read_excel('/Volumes/Elements/khartless/Projects/Data/Upton.xlsx',
                        sheet_name = 'Production',
                        index_col = [0])



In [31]:
safety_stock = 1500
unit_carrying_cost = .015
warehouse_capacity = 6000
initial_inventory = 2750

In [20]:
production_constraints

,Min_Production,Max_Production,Unit_Production_Cost,Unit_Carrying_Cost
Month,,,,
1,2000,4000,240,0.015
2,1750,3500,250,0.015
3,2000,4000,265,0.015
4,2250,4500,285,0.015
5,2000,4000,280,0.015
6,1750,3500,260,0.015


In [22]:
demand = pd.read_excel('/Volumes/Elements/khartless/Projects/Data/Upton.xlsx',
        sheet_name = 'Demand',
        index_col = [0])

In [23]:
demand

,Demand
Month,
1,1000
2,4500
3,6000
4,5500
5,3500
6,4000


In [29]:
production = LpVariable.dicts("production",
                            (list(production_constraints.index)),
                            lowBound=0,
                            cat='Integer')

In [30]:
model = LpProblem("Upton_cost_minimizing", LpMinimize)

In [ ]:
model += lpSum([production[month] * production_constraints.loc[month, 'Unit_Production_Cost'] for month in production_constraints.index]
        + [.5*unit_carrying_cost*production_constraints.loc[month, 'Unit_Production_Cost']*((2*production[month]))])